In [1]:
from hsmm_core.data_utils import DataLoader, TradingHours
from hsmm_core.feature_spaces import hmm_features
from hsmm_core.hsmm_runner import HmmCalibration
import time
from hsmm_core.consts import InitialisationMethod
from hsmm_core.data_utils import TradingHours, DataLoader
from hsmm_core.labelling import DataLabellingSimple
from hsmm_core.consts import ThresholdMethod, LabellingChoice
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import precision_recall_fscore_support
sc = StandardScaler()
import os
import pickle

In [2]:
##useful functions
def fwd_dates(_dates_list, _key_date):
    # returns a list of dates that are forward from the key_date
    fwd_dates_list = [i for i in _dates_list if i > _key_date]
    return fwd_dates_list
def remove_nans(features_tuple, labels, idx=1):
    # not the cleanest but useful
    # function to clean up nans as I seem to use it a lot, so better to have one function
    # combines the features and labels and removes rows with nans across so we dont lose the ordering
    # returns features and labels
    features_df = pd.concat([features_tuple[0], features_tuple[1], features_tuple[2], \
                             features_tuple[3]], axis=1, sort=False)
    labels_only = labels.drop(columns=['ReturnTradedPrice', 'Duration', 'states', 'TradedTime',
                                       'TradedPrice'], axis=1)
    df_concat = pd.concat([features_df, labels_only.iloc[:, 0:idx]], axis=1, sort='False')
    # only using 1st set of labels- but we can re-write this a bit
    df_x_nan = df_concat.dropna()  # dropping all nans
    label_column_loc_ = df_x_nan.shape[1] - 1  # location of labels column in the clean df
    labels_ = df_x_nan.iloc[:, label_column_loc_:label_column_loc_ + 1]  # keep pure labels
    features_ = df_x_nan.drop(df_x_nan.columns[label_column_loc_], axis=1)  # keeping the features only
    return features_, labels_ 

def prec_recall_report(y_true, y_predict):
    # function to ge the sci-kit learn classification metrics into a pretty DF for csv!
    report = pd.DataFrame(list(precision_recall_fscore_support(y_true, y_predict)),
                          index=['Precision', 'Recall', 'F1-score', 'Support']).T
    # Now add the 'Avg/Total' row
    report.loc['Avg/Total', :] = precision_recall_fscore_support(y_true, y_predict, average='weighted')
    report.loc['Avg/Total', 'Support'] = report['Support'].sum()
    return report


In [3]:
# finance_data=os.getenv('FINANCE_DATA')
# os.listdir(finance_data)

In [5]:

ticker= 'os'
###sort out the data###
data_symbols=os.listdir(os.getenv('FINANCE_DATA'))
synt_data_files=os.listdir(os.path.join(os.getenv('FINANCE_DATA'), ticker)) #unlabelled data
ticker_dir = os.path.join(os.getenv('FINANCE_DATA'),ticker)
# test_file_name =os.path.join(ticker_dir,synt_data[0])
synt_data_dates=[os.path.splitext(file_name)[0] for file_name in synt_data_files]
synt_data_dates.sort()#sort the dates. no special reason makes it cleaner to play with the indexation


In [6]:
fin_data_dir = os.path.join(os.path.expanduser("~"), 'FinData')
total_path = os.path.join(fin_data_dir, ticker)

In [12]:
total_path

'/home/ak/FinData/SYNT_2states'

In [13]:
from hsmm_core.data_utils import *


In [14]:
get_files_for_ticker(ticker)

['20160601',
 '20160602',
 '20160603',
 '20160606',
 '20160607',
 '20160608',
 '20160609',
 '20160610',
 '20160613',
 '20160614',
 '20160615',
 '20160616',
 '20160617',
 '20160620',
 '20160621',
 '20160622',
 '20160623',
 '20160624',
 '20160627',
 '20160628',
 '20160629',
 '20160630',
 '20160701',
 '20160704',
 '20160705',
 '20160706',
 '20160707',
 '20160708',
 '20160711',
 '20160712',
 '20160713',
 '20160714',
 '20160715',
 '20160718',
 '20160719',
 '20170601',
 '20170602',
 '20170605']

In [37]:

data_dir = os.getenv('FINANCE_DATA')

labels_path = os.path.join(data_dir, 'features_models/labels')

ticker_labels_path = os.path.join(labels_path, ticker + '/NON_DIRECTIONAL')

if not os.path.exists(os.path.join(data_dir, ticker)):
    os.makedirs(os.path.join(data_dir, ticker))

if not os.path.exists(ticker_labels_path):
    os.makedirs(ticker_labels_path)

# example=pd.read_csv(test_file_name, index_col=0)

In [25]:
ticker_labels_path

'/home/ak/FinData/features_models/labels/SYNT_2states/NON_DIRECTIONAL'

In [38]:
n_hidden_states = 3

data_loader_init = {
    'trading_hours_filter': TradingHours.only_mkt_hours,
}


init_params = {
    'obs_model_name': 'ExpIndMixDiracGauss',
    'em_obs_init_method': InitialisationMethod.cluster,
    'em_hidden_init_method': InitialisationMethod.uniform,
    'no_hidden_states': n_hidden_states,
    'update_tag': 'tpsml'
}
sd = synt_data_dates[0]
ed = synt_data_dates[-1]

data_loader = DataLoader(**data_loader_init)

data_loader_hash = data_loader.data_loader_hash()

data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)

hmm_calibration_engine = HmmCalibration(init_params=init_params)
# hmm_calibration_engine.run_calibration_all_data(ticker, data,  data_loader_hash,
#                                                 force_recalc=True, use_multiprocessing=False,
#                                                 n_processes=2)
# features_engine = hmm_features()


In [29]:
ticker_labels_path

'/home/ak/FinData/features_models/labels/SYNT_2states/NON_DIRECTIONAL'

# create labels###

In [30]:
trading_hours_filter = TradingHours.only_mkt_hours


window = 25
threshold = 0.1

labelling_method_params = [{

    'labelling_method': LabellingChoice.price_move_in_window,
    'rolling_window': window,
    # Uncomment below if you want to check a price move only above a certain level
    'updown_threshold': threshold,  # this is multiplied by 100
    'threshold_method': ThresholdMethod.arbitrary,
}]

data_loader = DataLoader(trading_hours_filter)

data = data_loader.load_trades_data(ticker, start_date=sd, end_date=ed)

for label_init in labelling_method_params:
    print label_init
    labeller = DataLabellingSimple(label_init)
    labeller.label_training_data(data)

# SAVE THE DATA WITH LABELS 
for date, date_data in data.iteritems():
    date_data.to_csv(os.path.join(ticker_labels_path, str(date)+'.csv'))

{'rolling_window': 25, 'labelling_method': 'PrMov', 'updown_threshold': 0.1, 'threshold_method': 'arbitrary'}


In [31]:
# seq_clf = "_".join(("synt_model",  str(date), labels_clean.columns.values[0], "clfs", ".pickle"))
test_date=data.keys()[2]
stored_hmm, _ = hmm_calibration_engine.get_calibrated_hmm(ticker, test_date, data_loader_hash)
features_engine.hmm = stored_hmm

In [32]:
oos_dates= fwd_dates(_dates_list=data.keys(), _key_date=data.keys()[2])
oos_date=oos_dates[5]
features_load = features_engine.generate_features(data[oos_date])
labels_load = pd.read_csv(os.path.join(ticker_label_data, str(oos_date)+'.csv'), index_col=0)
test_x, test_y =remove_nans(features_load, labels_load)
test_x.shape, test_y.shape

((4834, 15), (4834, 1))

In [35]:
os.listdir(os.path.join(models_path, ticker))

[]

In [42]:
features_engine = hmm_features()

for date, date_data in data.iteritems():
    stored_hmm, _ = hmm_calibration_engine.get_calibrated_hmm(ticker, date, data_loader_hash)
    #load classifiers
    features_engine.hmm = stored_hmm
#     oos_dates= fwd_dates(_dates_list=data.keys(), _key_date=date)
#     for oos_date in oos_dates:
#         file_name = "_".join(('synt_model',str(oos_date)))
#         models_list=os.listdir(ticker_models_path)
#         clf_model=[s for s in models_list if file_name in s][0]
      
#      #         features_load = features_engine.generat(e_features(data[oos_date])
#         labels_load = pd.read_csv(os.path.join(ticker_label_data, str(oos_date)+'.csv'), index_col=0)
#         features, labels_clean = remove_nans(features_load, labels_load)
#         x_std = sc.fit_transform(features.values.astype(np.float))  # fit & transform the features
#         X_train, X_test, y_train, y_test = train_test_split(
#             x_std, labels_clean, test_size=0.01, random_state=1, stratify=labels_clean)  # probably can get rid of this
        


In [43]:
print stored_hmm

In [ ]:
date= data.keys()[-1]
oos_dates= fwd_dates(_dates_list=data.keys(), _key_date=date)
for oos_date in oos_dates:
    file_name = "_".join(('synt_model',str(oos_date)))
    models_list=os.listdir(ticker_models_path)
    clf_model=[s for s in models_list if file_name in s][0]
    best_clf = pickle.load(open(os.path.join(ticker_models_path,clf_model),"rb"))
    svc=best_clf['SVC']
    features_load = features_engine.generate_features(data[oos_date])
    labels_load = pd.read_csv(os.path.join(ticker_label_data, str(oos_date)+'.csv'), index_col=0)
    features, labels_clean = remove_nans(features_load, labels_load)
    x_std = sc.fit_transform(features.values.astype(np.float))  # fit & transform the features
    X_train, X_test, y_train, y_test = train_test_split(
        x_std, labels_clean, test_size=0.01, random_state=1, stratify=labels_clean)  
    y_true = labels_clean
    y_predict_clf= svc.predict(X_train)
    classif_rate = np.mean(y_predict_clf.ravel()== y_true.values.ravel())*100
    print ("Classification Rate for %s: %f" % (clf_model,classif_rate))

        


In [ ]:
svc.predict(X_train)

In [ ]:

#data_for_features = dates_for_features[i],
for index, date in enumerate(synt_data_dates):
    seq = (synt_data_dates[index], "csv"); # This is sequence of strings.
#     print str(".".join( seq ))
    input_date_data = pd.read_csv(os.path.join(ticker_dir, str(".".join(seq))))
    features_oos=features_engine.generate_features(input_date_data)
